In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import concurrent.futures

## Group Assignment
### Team Number: 4
### Team Member Names: Jacky Xu, Esha Kumar, Jingyi Fang
### Team Strategy Chosen: SAFE

In [2]:
# read the ticker file
tickers = pd.read_csv("Tickers.csv",header=None)
# rename the column as "ticker"
tickers.columns=['ticker']
# create a list that contains all the tickers
ticker_lst = tickers['ticker'].tolist()


In [3]:
# call_tickers takes a list of tickers and return a list of called tickers
def call_tickers(lst):
    # create an empty list to store the called tickers
    called_tickers= []
    # iterate the ticker list
    for i in lst:
        # call each ticker and append it to the called ticker list
        called_tickers.append(yf.Ticker(i))
    # return the called ticker list when finish
    return called_tickers

# get_ticker_info takes a called ticker, returns a list of ticker name, currency, average daily volumn and closing price on Nov 26
def get_ticker_info(i):
    # extract the ticker name
    ticker_name = str(i)[24:-1]
    # use try and except to ignorethe delisted stocks
    try:
        # extract the currency, average daily volumn and closing price on 2021-11-26
        currency =  i.info['currency']
        avg_daily_volumn = i.history(start = "2021-07-02", end = "2021-10-22" ,interval='1d')['Volume'].mean()
        price1126 = i.history(start = '2021-11-24', end = '2021-11-25',interval='1d')['Close'][0]
        # return a list with the information 
        return [ticker_name,currency,avg_daily_volumn,price1126]
    except:
        pass


# get_prices takes a list of called tickers, start date adn end date, returns the historical prices in the given time period
def get_prices(lst,start,end):
    # open an empty dataframe
    prices = pd.DataFrame()
    # iterate the ticker list
    for i in lst:
        # extract the historical closing prices and store into the dataframe
        prices[str(i)[24:-1]] = i.history(start = start,end=end,interval = '1d')['Close']
    # return the prices
    return prices

# get_weekly_returns takes the historical prices(or portfolio value) and returns the weekly returns
# usually we use monthly returns but since the competition only lasts for 6 days, we decided to use the weekly returns
def get_weekly_returns(prices):
    weekly_returns = prices.resample('W').first().pct_change()
    weekly_returns = weekly_returns.iloc[1:]
    return weekly_returns

# we still use monthly returns to calculate the beta and others
def get_monthly_returns(prices):
    monthly_returns = prices.resample('MS').first().pct_change()
    #monthly_returns = monthly_returns.iloc[1:]
    return monthly_returns
    
# get_std takes the prices of the stock and returns the std of the prices
def get_std(prices):
    # open an empty dataframe with column names set up
    std = pd.DataFrame(columns = ('ticker','std'))
    # set the index
    index = 0
    # iterate through the ticker names
    for i in prices.columns:
        # store the ticker name and the std of prices
        std.loc[index] = [i, prices[i].std()]
        # add 1 to the index
        index += 1
    # return the std
    return std


# rank takes a dataframe, column name and method and rank the dataframe by the given column using the given method
def get_rank(df,column_name,method):
    # use the build-in rank function to rank the dataframe, store the rankings in a new column
    df[column_name+'_rank'] = df[column_name].rank(method=method)
    # return the ranked dataframe
    return df

# get_std_beta_rank takes the ranked std dataframe and the ranked beta dataframe, rank the dataframe by both and return the top 40 stocks
def get_std_beta_rank(std,beta):
    # combine the std dataframe and the beta dataframe
    rank = pd.concat([std,beta['beta'],beta['beta_rank']],join='inner',axis=1)
    # add up the ranks to get the final rank
    rank['final_rank'] = rank['std_rank']*8 + rank['beta_rank']
    # rank by the final rank
    rank = rank.sort_values(by='final_rank')
    # takes the top 40 stocks
    rank = rank[:40]
    # return the dataframe
    return rank



In [4]:
# Calculating Beta
def get_beta(good_tickers, prices, start_date, end_date, ticker_symbols):
    Ticker = '^GSPC'
    MarketIndex = yf.Ticker(Ticker) # The symbol yfinance uses for the S&P 500

    MarketIndex_hist = MarketIndex.history(start=start_date, end=end_date)

    # DataFrame for Market Index
    marketDF = pd.DataFrame(MarketIndex_hist['Close'])
    marketDF.columns = [Ticker]
    

    # Loop iterates through the column of prices
    betaList = []
    for i in range(len(ticker_symbols)):
        currentStockPrice = prices[ticker_symbols[i]]
        currentStockPrice = pd.concat([currentStockPrice, marketDF], join = 'inner', axis=1)
        
        # Getting the Monthly Return of each stock
        monthly_returns = currentStockPrice.resample('MS').first().pct_change()  # Dropping the first entry (since it's N/A)
        monthly_returns.drop(index=monthly_returns.index[0], inplace=True)
        
        
        # Calculate the market variance (you will need to reference the column correponding to the market)
        MarketVar = monthly_returns[Ticker].var()
        
        betaList.append(monthly_returns.cov() / MarketVar)
        
    # Filter through betaList and extract the beta for each stock
    for i in range(len(betaList)):
        betaList[i] = betaList[i].iat[0,1]
    
    # Creating a DataFrame for Tickers and their Beta value
    beta = pd.DataFrame(columns = ('ticker','beta'))
    for i in range(len(good_tickers)):
        beta.loc[i] = [str(good_tickers[i])[24:-1], betaList[i]]

    return beta

In [5]:
# get_correlation takes a ticker(self), calculate and rank the correlations between self and the rest of the tickers in the given dataframe(std_rank)
def get_correlation(self,std_rank,prices,length):
    # open an empty dataframe
    rank = pd.DataFrame()
    # get the weekly return of self
    self_weekly_returns = pd.DataFrame(get_weekly_returns(prices[self]))
    # iterate rest of the tickers
    for i in range(1,length):
        # get the ticker name and the weekly returns
        candidate = std_rank.index[i]
        candidate_weekly_returns = pd.DataFrame(get_weekly_returns(prices[candidate]))
        # calculate the correlations
        corr = self_weekly_returns[self].corr(candidate_weekly_returns[candidate])  
        # store the correlation
        rank[candidate] = [corr]
    # return the transposed dataframe
    return rank.T

# pair_by_corr takes the ranked dataframe and the prices of the stocks, 
# returns 10 portfoliois in one dataframe, each with a pair of stocks with the lowest correlation
def pair_by_corr(rank, prices):
    # open an empty dataframe
    pairs = pd.DataFrame()
    # counter is the number of pairs
    counter = 0
    # loop until we have 10 pairs
    while counter < 10:
        # self is the stock that has lowest std and beta(which is the safest one)
        self = rank.index[0]
        # get the weekly_returns for self
        self_weekly_returns = pd.DataFrame(get_weekly_returns(prices[self]))
        # for all other stocks in the ranked dataframe
        corrs = get_correlation(self,rank,prices,len(rank)-1)
        corrs.columns = ['corr']
        corrs = corrs.sort_values(by='corr')
        champion = corrs.index[0]
        champion_weekly_returns = pd.DataFrame(get_weekly_returns(prices[champion]))
        # get current price from the ticker info dataframe
        self_price = prices[self][0]
        champion_price = prices[champion][0]
        # calculate the number of shares we could purchase using the current price
        self_shares = 100000 /  self_price
        champion_shares = 100000 /  champion_price
        # get weekly price, portfolio value, and portfolio return for self and champion
        pairs[self+'_price'] = prices[self].resample('W').first()
        pairs[self+'_value'] =  pairs[self+'_price']*self_shares
        pairs[self+'_return'] = pd.DataFrame(get_weekly_returns(pairs[self+'_value']))
        pairs[champion+'_price'] = prices[champion].resample('W').first()
        pairs[champion+'_value'] =  pairs[champion+'_price']*champion_shares
        pairs[champion+'_return'] = pd.DataFrame(get_weekly_returns(pairs[champion+'_value']))
        # remove self and champion from the ranked dataframe to prepare for the next iteration
        rank = rank[1:]
        rank = rank[rank.index != str(champion)]
        # add 1 to the counter for each finished pair
        counter += 1
    # return the pairs
    return pairs

# get_difference calculate the max difference betetween the given portfolio value and $100000
def get_difference(df): 
    return max(abs(df['value'].max()-100000) , abs(df['value'].min()-100000))

# when 'stock' is True, create_portfolio takes two stocks and the pairs dataframe, 
# returns a weight and the weighted portfolio with the lowest sharpe ratio
# when 'stock' is False, create_portfolio takes two portfolio(old and new), the min and max range for the sharpe ratio,
# returns a weight and the weighted portfolio with the lowest sharpe ratio
def create_portfolio(range_min,range_max,stock1,stock2,pairs):
    # open an empty portfolio 
    candidate = pd.DataFrame()
    champion = 0
    # min_ratio stores the lowest sharpe ratio
    min_difference = ['+' , 100000]
    # weight_1 is the weight of stock1 or the portfolio weight for old
    weight_1 = 0
    # iterates the given range
    for i in range(range_min,range_max+1):
        # get the portfolio value of each pair by multiplying the weight and the value of each stock
        # add them up and double the result(since each stock value starts from 500000)
        candidate['value'] = pairs[stock1+'_value']*(i/100)+pairs[stock2+'_value']* (1-i/100)
        # calculate the max difference between the portfolio value and 100000
        difference = get_difference(candidate)
        # if the absolute value of the difference(since the ratio could be negative) is less than the min_difference
        if abs(difference) <= min_difference[1]:
            # if the portfolio is roughly greater than 100000, we assume that the difference is positive
            if candidate['value'].median() > 100000:
                # replace the min_difference with difference
                 min_difference = ['+',abs(difference)]
            # otherwise it's negative
            else:
                min_difference = ['-',abs(difference)]
            # update the champion and the weight
            champion = candidate
            weight_1 = i
    # return the optimal weight, minimal difference and the portfolio with the optimal weight
    return weight_1, min_difference, champion


In [6]:
# set up the start date and the end date
start_date = "2021-07-02"
end_date = "2021-10-22" 

# call the tickers
called_tickers = call_tickers(ticker_lst)

In [7]:
# open an empty dataframe and a list to store the ticker info and the tickers
ticker_info = pd.DataFrame()
tickers = []
# Using threading to run the get_ticker_info function, store the result
with concurrent.futures.ThreadPoolExecutor() as executor:
    result = executor.map(get_ticker_info, called_tickers)
# iterates the results
for i in result:
    try:
        # filter the USD tickers
        if i[1] == 'USD':
            # filter the average daily volumn
            if i[2] > 10000:
                # store the eligible ticker and its closing price on 2021-11-26
                ticker_info[i[0]] = [i[3]]
    except:
        pass
# format the ticker_info dataframe
ticker_info = ticker_info.T
ticker_info.columns = ['11-26_price']

In [8]:
ticker_info

,11-26_price
AAPL,157.869995
ABBV,117.070000
ABT,127.639999
ACN,370.779999
AIG,56.119999
AMZN,3696.060059
AXP,176.210007
BA,227.250000
BAC,46.320000
BIIB,256.790009


In [9]:
# call the tickers again
good_tickers = call_tickers(ticker_info.index)

# extract the closing prices for the good tickers
prices = get_prices(good_tickers, start_date, end_date)

# Get a list of Ticker Symbols (list of Strings)
ticker_symbols = prices.columns

# calculate the std according to the closing prices
std = get_std(prices)

# rank them in ascending order
std = get_rank(std,'std','min')

In [10]:
# calculate the std according to the closing prices
std = get_std(prices)

# rank them in ascending order
std = get_rank(std,'std','min')

In [11]:
# calculate the beta according to the closing prices
beta = get_beta(good_tickers, prices, start_date, end_date, ticker_symbols)

# rank them in ascending order
beta = get_rank(beta,'beta','min')

# combine the rank of std and beta, rank the combined rank, then format the result dataframe
std_beta_rank = get_std_beta_rank(std,beta)
std_beta_rank.set_index('ticker',inplace=True)
std_beta_rank.head()

,std,std_rank,beta,beta_rank,final_rank
ticker,,,,,
MON,0.046874,1.0,-0.146849,13.0,21.0
T,0.539296,2.0,-0.460408,11.0,27.0
KMI,0.760351,3.0,-1.738541,5.0,29.0
KO,1.205896,5.0,1.013712,28.0,68.0
CVS,1.819270,6.0,0.826085,24.0,72.0


In [12]:
# get the pair portfolios
pairs = pair_by_corr(std_beta_rank,prices)
pairs.head()

,MON_price,MON_value,MON_return,PFE_price,PFE_value,PFE_return,T_price,T_value,T_return,NEE_price,...,USB_return,SO_price,SO_value,SO_return,C_price,C_value,C_return,PEP_price,PEP_value,PEP_return
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-04,9.655,100000.000000,NaN,39.030785,100000.000000,NaN,27.617552,100000.000000,NaN,73.615639,...,NaN,60.274460,100000.000000,NaN,69.385284,100000.000000,NaN,148.910004,100000.000000,NaN
2021-07-11,9.730,100776.797632,0.007768,38.598530,98892.528563,-0.011075,27.419138,99281.566120,-0.007184,74.458031,...,-0.027098,60.382214,100178.771828,0.001788,67.217293,96875.430207,-0.031246,149.130005,100147.741062,0.001477
2021-07-18,9.700,100466.080555,-0.003083,39.060253,100075.500753,0.011962,27.401449,99217.517173,-0.000645,74.418388,...,0.028031,60.538948,100438.806455,0.002596,68.429398,98622.349323,0.018033,149.509995,100402.921785,0.002548
2021-07-25,9.670,100155.363477,-0.003093,39.443394,101057.137602,0.009809,26.737581,96813.727166,-0.024227,76.301399,...,-0.053317,61.508747,102047.778222,0.016019,64.132851,92430.046485,-0.062788,155.800003,104626.955356,0.042071
2021-08-01,9.675,100207.151303,0.000517,41.074177,105235.334626,0.041345,27.112812,98172.394987,0.014034,76.222122,...,0.030637,62.429565,103575.487191,0.014971,66.783699,96250.522845,0.041334,157.070007,105479.822350,0.008152


In [13]:
# we now write a for loop to iterates the pairs to store  
# 1). the optimal weight of the pair portfolio, 
# 2). the ticker names,
# 3). the minimal difference between the portfolio value and $100000

# open empty lists for the protfolio weights, differences and symbols
portfolio_weights = []
diffs = []
stock1_symbols = []
stock2_symbols = []
# extract all ticker names from the columns of the pairs dataframe
symbols = pairs.filter(like='_price', axis=1).columns
# the number of pairs is the number of loop
num_of_pairs = round(len(symbols)/2)

# iterates the paris
for i in range(num_of_pairs):
    # get the ticker names for the pair of stocks, and append to the corresponding symbol lists
    stock1 = symbols[0][:-6]
    stock1_symbols.append(stock1)
    stock2 = symbols[1][:-6]
    stock2_symbols.append(stock2)
    
    # combine the pair of stock, store the optimal stock weight, difference, and the portfolio with optimal weight
    w1, diff, champion = create_portfolio(30,70,stock1,stock2,pairs)
    # append the result to their corresponding lists
    portfolio_weights.append(w1)
    diffs.append(diff)
    # remove the finished pair
    symbols = symbols[2:]


In [14]:
diffs

[['+', 7934.7127661386185],
 ['-', 2822.7058700563794],
 ['+', 3441.814844497363],
 ['+', 3075.308291349109],
 ['+', 4132.866476841271],
 ['-', 8070.687035911789],
 ['-', 5109.902850087121],
 ['+', 3059.7243755941163],
 ['+', 4758.556211509422],
 ['+', 4056.7734494834876]]

In [15]:
# decorate the differences with signs
for i in range(num_of_pairs):
    if diffs[i][0] == '+':
        diffs[i] = diffs[i][1]
    else:
        diffs[i] = (diffs[i][1])*-1
diffs

[7934.7127661386185,
 -2822.7058700563794,
 3441.814844497363,
 3075.308291349109,
 4132.866476841271,
 -8070.687035911789,
 -5109.902850087121,
 3059.7243755941163,
 4758.556211509422,
 4056.7734494834876]

In [16]:
# create a dataframe with the weights and the differences
weights = pd.DataFrame(portfolio_weights, diffs)
# format the dataframe, calculate the stock 2 weight and add the ticker names in
weights.reset_index(inplace=True)
weights.columns = ['diff','stock1_weight']
weights['stock1_symbol'] = stock1_symbols
weights['stock2_weight'] = 100 - weights['stock1_weight']
weights['stock2_symbol'] = stock2_symbols
weights
# separate the weights dataframe by signs
pos = weights[weights['diff'] > 0]
neg = weights[weights['diff'] < 0]
# calculate the sum of positive differences and negative diffrences
pos_diff = pos['diff'].sum()
neg_diff = neg['diff'].sum()
# calculate the total differences
total_diff = pos_diff - neg_diff
# calculate the weights of positve and negative diffserences
pos_weight = pos_diff/total_diff
neg_weight = (neg_diff/total_diff)*-1
print(pos_weight,neg_weight)
# get number of stocks
num_of_stocks = num_of_pairs*2
# each stock must make up a minimum of (100/(2n))% of the portfolio , we add/subtract 0.01 just in case, 
# then divided by 30, which the minimal weight of each stock in its local pair portfolio
min_weight = ((100/(2*num_of_stocks))+0.01)/30
# same for the maximal weight
max_weight = (35-0.01)/70

# our goal is to weight the positive differences and the negative differences to a 50-50 percentage
# so that they can cancel each other

# when positive differences is less than the negative difference, 
if pos_weight < neg_weight:
    # maximize the postive weight by subtracting the minimal negative weight from 1
    # we use minimal negative weight here instead of maximum positive weight because we expect to have 20 stocks
    # then each stock will be weighted 5% in average
    # it is very unlikely that a stock will be weighted more than 35%
    # but it is very likely that a stock will be weighted lower than 2.5% (if we do get 20 stocks)
    neg_weight = min_weight*len(neg)
    pos_weight = 1 - neg_weight
else:
    print(0.5/pos_weight)
    # same logic here
    pos_weight = min_weight*len(pos)
    neg_weight = 1 - pos_weight

pos_weight/len(pos), neg_weight/len(neg)

0.6555694254222423 0.3444305745777577
0.7626957277300686


(0.08366666666666665, 0.13811111111111116)

In [17]:
# apply the weights 
pos['stock2_weight']*= pos_weight/len(pos)
pos['stock1_weight']*= pos_weight/len(pos)
neg['stock2_weight'] *= neg_weight/len(neg)
neg['stock1_weight'] *= neg_weight/len(neg)
neg['stock2_weight'].sum()+neg['stock1_weight'].sum()+pos['stock2_weight'].sum()+pos['stock1_weight'].sum()

<ipython-input-17-463d51a2f70d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['stock2_weight']*= pos_weight/len(pos)
<ipython-input-17-463d51a2f70d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['stock1_weight']*= pos_weight/len(pos)
<ipython-input-17-463d51a2f70d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

100.00000000000001

In [18]:
# combine the pos and neg, as well as stock1 and stock2
weights = pd.concat([pos,neg],join='inner',axis=0)
weights_1 = weights[['stock1_weight','stock1_symbol']]
weights_2 = weights[['stock2_weight','stock2_symbol']]
weights_1.columns = ['weight','symbol']
weights_2.columns = ['weight','symbol']
weights = pd.concat([weights_1,weights_2],join = 'inner',axis = 0)
weights.reset_index(inplace = True)
weights.drop('index',axis = 1)

,weight,symbol
0,5.856667,MON
1,2.510000,KMI
2,4.685333,KO
3,5.605667,CVS
4,5.856667,CSCO
5,3.932333,USB
6,5.856667,C
7,9.529667,T
8,9.667778,MO
9,5.248222,SLB


In [19]:
neg

,diff,stock1_weight,stock1_symbol,stock2_weight,stock2_symbol
1,-2822.705870,9.529667,T,4.281444,NEE
5,-8070.687036,9.667778,MO,4.143333,COP
6,-5109.902850,5.248222,SLB,8.562889,PG


In [20]:
# initialize the final portfolio
final_portfolio = pd.DataFrame(columns = ['Ticker', 'Price', 'Shares', 'Value', 'Weight'])

In [21]:
# fill in the required information
for i in range(len(weights)):
    ticker = weights['symbol'][i]
    price = pairs[ticker+'_price'][0] #ticker_info.loc[ticker,'11-26_price']
    weight = weights['weight'][i]
    value = weight/100*100000
    shares = value/price
    final_portfolio.loc[i+1] = [ticker,price,shares,value,weight]

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [22]:
### Check the final portfolio, using the price of start date to buy shares
ticker = final_portfolio['Ticker']
ticker_lst = ticker.tolist()
shares = final_portfolio['Shares']
price = pairs[ticker+'_price']
for i in range(len(ticker_lst)):
    ticker_name = ticker_lst[i]
    price[ticker_name+'_price'] *= final_portfolio.loc[i+1,'Shares']

price['final'] = price.sum(axis = 1)
price.head()

<ipython-input-22-37b0c77de623>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price[ticker_name+'_price'] *= final_portfolio.loc[i+1,'Shares']
<ipython-input-22-37b0c77de623>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['final'] = price.sum(axis = 1)


,MON_price,KMI_price,KO_price,CVS_price,CSCO_price,USB_price,C_price,T_price,MO_price,SLB_price,...,ABT_price,MRK_price,ABBV_price,GM_price,SO_price,PEP_price,NEE_price,COP_price,PG_price,final
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-04,5856.666667,2510.000000,4685.333333,5605.666667,5856.666667,3932.333333,5856.666667,9529.666667,9667.777778,5248.222222,...,5856.666667,3681.333333,2761.000000,2510.000000,4434.333333,2510.000000,4281.444444,4143.333333,8562.888889,100000.000000
2021-07-11,5902.161115,2492.428836,4659.390219,5520.660150,5795.408891,3825.776930,5673.671029,9461.202313,9492.850072,4988.932003,...,5895.367226,3658.383479,2774.425110,2446.143147,4442.260672,2513.708301,4330.437491,3980.240992,8567.929570,98903.580010
2021-07-18,5883.963451,2503.241840,4711.276448,5564.183772,5822.756045,3933.016522,5775.982259,9455.098662,9659.641881,4996.934925,...,5894.870962,3632.155280,2819.974123,2510.425803,4453.791474,2520.113337,4328.131915,3948.547132,8641.020422,99567.021320
2021-07-25,5865.765788,2322.121703,4819.372758,5485.977230,5804.160180,3723.319132,5413.319722,9226.025486,9517.259342,4265.480850,...,5916.317107,3608.737451,2797.840530,2306.509547,4525.138646,2626.136579,4437.646775,3545.769209,8848.949983,97592.382173
2021-08-01,5868.798828,2409.978321,4934.387765,5669.719893,6067.786731,3837.389034,5637.072288,9355.502001,9684.050361,4715.236890,...,5952.674431,3617.635815,2855.290612,2374.197818,4592.882354,2647.543541,4433.036067,3783.792323,8894.458242,99972.840214


In [23]:
price['final']

Date
2021-07-04    100000.000000
2021-07-11     98903.580010
2021-07-18     99567.021320
2021-07-25     97592.382173
2021-08-01     99972.840214
2021-08-08    100025.656186
2021-08-15    100344.042444
2021-08-22    102059.598048
2021-08-29    101571.596573
2021-09-05    101776.529553
2021-09-12    101757.568635
2021-09-19    101469.462777
2021-09-26     99436.481916
2021-10-03    101421.664825
2021-10-10    100846.189189
2021-10-17    102014.436105
2021-10-24    102416.353301
Freq: W-SUN, Name: final, dtype: float64